In [3]:
'''
this program gives a report.csv file containing suspected numbers list, incoming and ougoing calls (counts 
and durations), total count and duration for a given number, how many IMEIs for a given number, how many calls
where first cell id and last cell id are same. Also you can generate the entire portion of cdr for 
suspected numbers.
this codes takes various inputs like threshold calls count and duration and can generate various types of
report based on requirements
To execute
copy this code and cdr file into
C:\\User\\Administrator or the default workspace of your compiler //report files are also creted here only//

to read the cdr file enterfile name in place of cdr1.csv in 3rd row  (hf=pd.read_csv('CDR1.csv))

set header=n where n is the row no.starting from 0, where columns names are there in the file
. In this case it is 3. 

For any help conatct Sumit Singh - 8700470095, sumit.singh53@gov.in'''

import pandas as pd   #importing the modules
import numpy as np

                                     
hf1=pd.read_csv('bsnl1.csv',header=0,dtype=object)
hf2=pd.read_csv('bsnl2.csv',header=0,dtype=object)
hf3=pd.read_csv('bsnl3.csv',header=0,dtype=object)
hf=pd.concat([hf1,hf2,hf3],ignore_index=True)
hf=hf.drop_duplicates()
df=hf[['Calling_Party','Duration','First_Cell_ID','Last_Cell_ID','Call_Type','IMEI']].copy()
column=['CALLING_NO','DURATION','FIRST_CELL','LAST_CELL','CALL_TYPE','IMEI']
df.columns=column

df.CALLING_NO=pd.to_numeric(df.CALLING_NO,errors='coerce')
df.DURATION=pd.to_numeric(df.DURATION,errors='coerce')
df.IMEI=pd.to_numeric(df.IMEI,errors='coerce')
df.FIRST_CELL=pd.to_numeric(df.FIRST_CELL,errors='coerce')
df.LAST_CELL=pd.to_numeric(df.LAST_CELL,errors='coerce')

df=df.replace({'CALL_TYPE':{'OUT_VOICE':1 ,'IN_VOICE':2,'OUT_SMS':3,'IN_SMS':4}})
df=df[((df['CALL_TYPE']==1)|(df['CALL_TYPE']==2))]
df=df.fillna(0)
df.index=range(df.CALLING_NO.count())
df=df.applymap(int)

print('input the lower calls count threshold')
a=int(input())
print('input the upper calls count threshold')
b=int(input())
print('input the lower total duration(in s) of all calls  threshold')
c=int(input())
print('input the upper total duration(in s) of all calls  threshold')
d=int(input())
print('input the lower multiple IMEI count threshold')
e=int(input())
print('input the upper multiple IMEI count  threshold')
f=int(input())
print('do you want report only where outgoing calls count is greater than incoming calls count? \nif yes, enter 1   see report_filtered.csv\n if you want report without this condition also enter 2 \notherwise enter any number')
g=int(input())
print('do you want complete portion of excel file for the the suspected numbers? \n if yes, enter 1 see sus_no_cdr.csv\n otherwise enter any number')
h=int(input())



df['COUNT']=df.groupby('CALLING_NO')['CALLING_NO'].transform('count')
df['COUNT_IN']=df.where(df.CALL_TYPE==2).groupby('CALLING_NO')['CALLING_NO'].transform('count').fillna(0)
df['DUR_IN']=df.where(df.CALL_TYPE==2).groupby('CALLING_NO')['DURATION'].transform('sum').fillna(0)
df['COUNT_OUT']=df.where(df.CALL_TYPE==1).groupby('CALLING_NO')['CALLING_NO'].transform('count').fillna(0)
df['DUR_OUT']=df.where(df.CALL_TYPE==1).groupby('CALLING_NO')['DURATION'].transform('sum').fillna(0)
df['DUR_SUM']=df.groupby('CALLING_NO')['DURATION'].transform('sum')
df['SAME_CELL']=df.where(df.FIRST_CELL==df.LAST_CELL).groupby('CALLING_NO')['CALLING_NO'].transform('count').fillna(0)
df['multiple_IMEI_count']=df.groupby('CALLING_NO')['IMEI'].transform('nunique')

df1=df[((df['DUR_SUM']>c)&(df['DUR_SUM']<d)&(df['COUNT']>a)&(df['COUNT']<b)&(df['multiple_IMEI_count']>e)&(df['multiple_IMEI_count']<f))]

df2=df1[['CALLING_NO','COUNT','COUNT_IN','COUNT_OUT','DUR_SUM','DUR_IN','DUR_OUT','SAME_CELL','multiple_IMEI_count']].copy()
cols=['sus_no','COUNT','COUNT_IN','COUNT_OUT','DUR_SUM','DUR_IN','DUR_OUT','SAME_CELL','multiple_IMEI_count']
df2.columns=cols

df2.index=range(df2.sus_no.count())
df2=df2.drop_duplicates()
df2.index=range(df2.sus_no.count())

df2=df2.groupby('sus_no',as_index=False).agg('max')






if(g==1):
    df3=df2[df2['COUNT_OUT']>df2['COUNT_IN']]
    df3.index=range(df3.sus_no.count())
    df3.to_csv('report_filtered.csv')# report where outgoing call count is greater than incoming call count
elif(g==2):
    df3=df2[df2['COUNT_OUT']>df2['COUNT_IN']]
    df3.index=range(df3.sus_no.count())
    df3.to_csv('report_filtered.csv')
    df2.to_csv('report.csv')
    
else:
    df2.to_csv('report.csv')



if(h==1):
    hf.CALLING_NO=pd.to_numeric(hf.CALLING_NO,errors='coerce')
    df4=hf.loc[hf['CALLING_NO'].isin(df2.sus_no)]
    df4.to_csv('sus_no_cdr.csv')
    


input the lower calls count threshold
10
input the upper calls count threshold
5000
input the lower total duration(in s) of all calls  threshold
0
input the upper total duration(in s) of all calls  threshold
50000
input the lower multiple IMEI count threshold
20
input the upper multiple IMEI count  threshold
500
do you want report only where outgoing calls count is greater than incoming calls count? 
if yes, enter 1   see report_filtered.csv
 if you want report without this condition also enter 2 
otherwise enter any number
2
do you want complete portion of excel file for the the suspected numbers? 
 if yes, enter 1 see sus_no_cdr.csv
 otherwise enter any number
0


In [2]:
df2[(df2['multiple_IMEI_count']>20)&(df2['multiple_IMEI_count']<100)]

,sus_no,COUNT,COUNT_IN,COUNT_OUT,DUR_SUM,DUR_IN,DUR_OUT,SAME_CELL,multiple_IMEI_count
0,401,75,75.0,0.0,32226,32226.0,0.0,10.0,47
1,501,90,90.0,0.0,26981,26981.0,0.0,22.0,66
2,4011,100,100.0,0.0,32320,32320.0,0.0,5.0,64
3,4044,37,37.0,0.0,20053,20053.0,0.0,3.0,32
4,5044,51,51.0,0.0,18677,18677.0,0.0,10.0,40
5,5065,69,69.0,0.0,15950,15950.0,0.0,12.0,51
6,8265,111,111.0,0.0,43707,43707.0,0.0,19.0,77
7,123456,65,65.0,0.0,28513,28513.0,0.0,6.0,60
11,1206695903,57,57.0,0.0,3287,3287.0,0.0,12.0,48
14,1409190607,33,33.0,0.0,3166,3166.0,0.0,2.0,32


In [79]:
lf=df4[(df4['multiple_IMEI_count']>20)]

In [10]:
hf3.head()

,Calling_Party,Called_Party,LRN,Date,Time,Duration,First_Cell_ID,Last_Cell_ID,Call_Type,IMEI,IMSI,SMS_Center_Number,Network
0,9940227915,9445779242,NaN,31-Aug-2018,00:00:03,0,4046423011652,NaN,IN_SMS,866255031293630,404805168409448,9840011824,40464
1,BW-CANBNK,9442267218,NaN,31-Aug-2018,00:00:07,0,40464233737181,NaN,IN_SMS,356833025383010,404806100104071,9417899997,40464
2,BZ-REDBUS,9442105027,NaN,31-Aug-2018,00:00:07,0,40464320557643,NaN,IN_SMS,867287032831930,404805176471428,9423096985,40464
3,BT-BEESMS,9487803039,NaN,31-Aug-2018,00:00:10,0,4046423099192,NaN,IN_SMS,867290027188600,404805175284629,9442599997,40464
4,BT-BSNLIN,9444155386,NaN,31-Aug-2018,00:00:12,0,40464233131333,NaN,IN_SMS,911565303139620,404801101018569,9442399997,40464


In [100]:

lf=df4[(df4['multiple_IMEI_count']>20)]
lf[['sus_no','COUNT','COUNT_OUT','multiple_IMEI_count','DUR_OUT']]

hf.Calling_Party=pd.to_numeric(hf.Calling_Party,errors='coerce').fillna(0)

hf.IMEI=pd.to_numeric(hf.IMEI,errors='coerce').fillna(0)


tf=hf[['Calling_Party','IMEI']].copy()
tf=tf[tf['Calling_Party'].isin(lf['sus_no'])]
tf=tf.applymap(int)

tf=tf[['Calling_Party','IMEI']].drop_duplicates()
tf.index=range(tf.Calling_Party.count())
tf.to_csv('imei_report.csv')
tf

,Calling_Party,IMEI
0,9445030533,359959083851070
1,9498100157,354020091543030
2,9498100157,352314071377960
3,9498100157,863176031099120
4,9498100157,869162022123240
5,9445190211,354490096528650
6,9445190211,359708052770240
7,9445190211,357976098314730
8,9498100157,869730034773600
9,9445190211,356565066982850


In [88]:
lf

,sus_no,COUNT,COUNT_IN,COUNT_OUT,DUR_SUM,DUR_IN,DUR_OUT,SAME_CELL,multiple_IMEI_count
2837,9442810949,46,0.0,23.0,4332,0.0,2113.0,0.0,21.0
15206,9445030533,62,0.0,34.0,3755,0.0,1884.0,45.0,23.0
15958,9445190211,66,0.0,29.0,1781,0.0,799.0,0.0,31.0
21053,9498100157,59,0.0,32.0,2431,0.0,1390.0,0.0,21.0
21571,9498113705,52,0.0,26.0,3399,0.0,1688.0,7.0,24.0
22212,9498129555,58,0.0,30.0,4087,0.0,2095.0,0.0,22.0
22734,9498136539,54,0.0,30.0,2138,0.0,1199.0,35.0,21.0
23853,9498154247,64,0.0,33.0,4140,0.0,2403.0,0.0,22.0


In [97]:
(tf.IMEI.value_counts()).count()

194

304497

In [98]:
lf.multiple_IMEI_count.sum()

185.0

In [74]:
df=df[((df['CALL_TYPE']==1)|(df['CALL_TYPE']==2))]

In [59]:

hf.Calling_Party=pd.to_numeric(hf.Calling_Party,errors='coerce').fillna(0)

hf.IMEI=pd.to_numeric(hf.IMEI,errors='coerce').fillna(0)


tf=hf[['Calling_Party','IMEI']].copy()
tf=tf[tf['Calling_Party']==8048767281]

In [60]:
tf=tf.applymap(int)
tf[['Calling_Party','IMEI']]

,Calling_Party,IMEI


In [31]:
df['multiple_IMEI_count']=df.groupby('CALLING_NO')['IMEI'].transform('nunique')

In [89]:
type(hf.Duration[2])

str

In [91]:
hf.Duration[3]

'0'

In [18]:
(tf.IMEI.value_counts()).count()

154

In [4]:
df.IMEI.count()

851291

In [70]:
df.CALLING_NO[11]

0

In [103]:
df.CALL_TYPE.value_counts()

2    392966
1    327763
Name: CALL_TYPE, dtype: int64